In [1]:
import torch
from torch import nn
import os
import pickle


In [2]:
from vb import CustomDataset

In [3]:
# train_dataset = CustomDataset(train=True, target_len=2**13)
# test_dataset = CustomDataset(train=False, target_len=2**13, fixed=True)


In [4]:

# with open('train_dataset.pkl', 'wb') as f:
#     pickle.dump(train_dataset, f, pickle.HIGHEST_PROTOCOL)

# with open('test_dataset.pkl', 'wb') as f:
#     pickle.dump(test_dataset, f, pickle.HIGHEST_PROTOCOL)


In [4]:
with open('train_dataset.pkl', 'rb') as f:
    train_dataset = pickle.load(f)
with open('test_dataset.pkl', 'rb') as f:
    test_dataset = pickle.load(f)

In [5]:
from torch import nn
import torch


class CBP(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(CBP, self).__init__()
        self.conv = nn.Conv1d(in_channels, out_channels, 3, 1, padding=1)
        self.bn = nn.BatchNorm1d(out_channels)
        self.prelu = nn.PReLU()
    def forward(self, x, residual=None):
        x = self.conv(x)
        x = self.bn(x)
        if residual==None:
            x = self.prelu(x)
        else:
            x = self.prelu(x+residual)
        return x
        
class Pool(nn.Module):
    def __init__(self, in_channels):
        super(Pool, self).__init__()
        self.conv = nn.Conv1d(in_channels, in_channels, 2, 2, padding=0)
        self.bn = nn.BatchNorm1d(in_channels)
        self.prelu = nn.PReLU()
        
    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = self.prelu(x)
        
        return x
        
        
class unPool(nn.Module):
    def __init__(self, in_channels):
        super(unPool, self).__init__()
        self.tconv = nn.ConvTranspose1d(in_channels, in_channels, 2, 2)
        self.bn = nn.BatchNorm1d(in_channels)
        self.prelu = nn.PReLU()
        
    def forward(self, x):
        x = self.tconv(x)
        x = self.bn(x)
        x = self.prelu(x)
        
        return x
          
        
        
        
        
        

class Encoder(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(Encoder, self).__init__()
        
        self.residual = nn.Conv1d(in_channels, out_channels, 1)  # Skip connection
        self.cbp11 = CBP(in_channels, out_channels)
        self.cbp12 = CBP(out_channels, out_channels)
        
        self.cbp21 = CBP(out_channels, out_channels)
        self.cbp22 = CBP(out_channels, out_channels)
        
        self.pool = Pool(out_channels)
        
        
    def forward(self, x):
        residual = self.residual(x)
        x = self.cbp11(x)
        residual = self.cbp12(x, residual)
        
        x = self.cbp21(residual)
        skip = self.cbp22(x, residual)
        
        x = self.pool(skip)
        
        return x, skip

    
    
    
    
    
class Decoder(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(Decoder, self).__init__()
        self.unpool = unPool(in_channels)
        self.residual = nn.Conv1d(in_channels, out_channels, 1)  # Skip connection
        
        self.cbp11 = CBP(in_channels*2, out_channels)
        self.cbp12 = CBP(out_channels, out_channels)
        
        self.cbp21 = CBP(out_channels, out_channels)
        self.cbp22 = CBP(out_channels, out_channels)
        
    def forward(self, x, skip):
        
    
        x = self.unpool(x)
        residual = self.residual(x)
        x = self.cbp11(torch.cat([x, skip], dim=1))
        residual = self.cbp12(x, residual)
    
        x = self.cbp21(residual)
        x = self.cbp22(x, residual)
        
        return x


class Feature_Encoder(nn.Module):
    def __init__(self):
        super(Feature_Encoder, self).__init__()
        # Encoder blocks
        self.encoder_block_1 = Encoder(1, 16)
        self.encoder_block_2 = Encoder(16, 32)
        self.encoder_block_3 = Encoder(32, 64)
        self.encoder_block_4 = Encoder(64, 128)
        self.encoder_block_5 = Encoder(128, 256)

        # Decoder blocks
        self.decoder_block_5 = Decoder(256, 128)
        self.decoder_block_4 = Decoder(128, 64)
        self.decoder_block_3 = Decoder(64, 32)
        self.decoder_block_2 = Decoder(32, 16)
        self.decoder_block_1 = Decoder(16, 16)
        
        self.conv1 = nn.Conv1d(16, 16, 3, 1, padding=1)  # Remove dilation
        self.conv2 = nn.Conv1d(16, 32, 3, 1, padding=1)  # Remove dilation
        self.conv3 = nn.Conv1d(32, 64, 3, 1, padding=1)
        self.conv4 = nn.Conv1d(64, 128, 3, 1, padding=1)
        self.conv7 = nn.Conv1d(128, 1, 1, 1)

        
        self.bn1 = nn.BatchNorm1d(16)
        self.bn2 = nn.BatchNorm1d(32)
        self.bn3 = nn.BatchNorm1d(64)
        self.bn4 = nn.BatchNorm1d(128)
        
        self.prelu1 = nn.PReLU()
        self.prelu2 = nn.PReLU()
        self.prelu3 = nn.PReLU()
        self.prelu4 = nn.PReLU()
        
        self.tanh = nn.Tanh()

    def forward(self, x):
        out, skip1 = self.encoder_block_1(x)
        out, skip2 = self.encoder_block_2(out)
        out, skip3 = self.encoder_block_3(out)
        out, skip4 = self.encoder_block_4(out)
        out, skip5 = self.encoder_block_5(out)
        
        out = self.decoder_block_5(out, skip5)
        out = self.decoder_block_4(out, skip4)
        out = self.decoder_block_3(out, skip3)
        out = self.decoder_block_2(out, skip2)
        out = self.decoder_block_1(out, skip1)

        out = self.conv1(out)
        out = self.bn1(out)
        out = self.prelu1(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out = self.prelu2(out)
        out = self.conv3(out)
        out = self.bn3(out)
        out = self.prelu3(out)
        out = self.conv4(out)
        out = self.bn4(out)
        out = self.prelu4(out)

        out = self.conv7(out)

        return self.tanh(out)

model = Feature_Encoder()
sample_input = torch.randn(64, 1, 8192)
model(sample_input)
trainable_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)
trainable_parameters

1860247

In [7]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True, pin_memory=True,drop_last=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False, pin_memory=True,drop_last=True)
device = 'cuda'

In [8]:

from torch.utils.tensorboard import SummaryWriter

device = 'cuda'
model = Feature_Encoder().to(device)
loss_fn = nn.MSELoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

writer = SummaryWriter('boards/1_8M_unet_tanh')
x_back = None
l_list = []
trainable_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)
print('trainable parameters:', trainable_parameters)
for epoch in range(0,100000000000):
    t_loss = 0
    model.train()
    for clean, noisy in train_loader:
        clean, noisy = clean.to(device), noisy.to(device)
        
        pred = model(noisy)
        
        loss = loss_fn(pred, clean)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        t_loss+=loss.item()
    # l_list.append(t_loss)
    t_loss /= len(train_loader)
    
    
    
    model.eval()
    with torch.no_grad():
        t_tloss = 0
        for clean, noisy in test_loader:
            clean, noisy = clean.to(device), noisy.to(device)
            pred = model(noisy)
            loss = loss_fn(pred, clean)
            t_tloss+=loss.item()
        # l_list.append(t_tloss)
        t_tloss /= len(test_loader)
        
    writer.add_scalars('run_14h', {'train_loss':t_loss,
                                        'test_loss':t_tloss}, epoch)
    writer.flush()
    if epoch%10==1:
        print(epoch, t_loss, t_tloss)
        try:
            os.mkdir("ckpts/1_8M_unet_tanh")
        except:
            pass
        torch.save(model.state_dict(), f'ckpts/1_8M_unet_tanh/ckpt_{epoch}_.pt')

writer.close()

trainable parameters: 1860247
1 0.009765791065163083 0.006257620290853083
11 0.001557788479840383 0.0011931456053086247
21 0.0010129221110320133 0.0005796091378821681
31 0.000732606990075308 0.00038247361226240173
41 0.0006182024648296647 0.0003080479024598996
51 0.0005485252194274734 0.0002661102128816613
61 0.0005177123934522064 0.0002384260684872667
71 0.0004943805924590884 0.00022900637729132237
81 0.0004717807602396028 0.00020546807354548946
91 0.00045328459107420513 0.00021793483877748562
101 0.00044585900743388467 0.00020674548795796
111 0.00043199921670343934 0.0001895040416760215
121 0.0004254064081275525 0.00017709269195620436
131 0.00042027140467932136 0.00018418588782272613
141 0.0004105527038013356 0.00016874173707037698
151 0.00040743052175255595 0.0001801531592112345
161 0.0004001797050780927 0.00016823881499779722
171 0.0003944739515847889 0.00016570800107729156
181 0.00038809189233385646 0.00017636065240367316
191 0.00037861013624933547 0.00015734054795757402


KeyboardInterrupt: 

In [8]:
import tqdm
import librosa
import torchaudio
import numpy as np
device = 'cuda'

with open("data/test_dataset.txt", 'r') as f:
    test_list = f.readlines()
    test_list = [x.strip() for x in test_list]

clean_path = "/workspace/nas-dataset/jis/voicebank_demand/clean_trainset_56spk_wav/"
noisy_path = "/workspace/nas-dataset/jis/voicebank_demand/noisy_trainset_56spk_wav/"
x1_path = "/workspace/nas-dataset/jis/VB_readymade/sample_enh/1_8M_unet/"
x2_path = "/workspace/nas-dataset/jis/VB_readymade/sample_enh/x2_1_8M_unet/"
try:
    os.mkdir(x2_path)
except:
    pass
model = model.to(device)
model.load_state_dict(torch.load('ckpts/1_8M_unet_tanh/ckpt_191_.pt'))
model.eval()
with torch.no_grad():
    for i, file in enumerate(tqdm.tqdm(test_list)):
        clean, sr1 = librosa.load(os.path.join(clean_path, file), sr=16000,res_type='kaiser_best')
        noisy, sr2 = librosa.load(os.path.join(noisy_path, file), sr=16000,res_type='kaiser_best')
        x1, sr2 = librosa.load(os.path.join(x1_path, file), sr=16000,res_type='kaiser_best')
        
        
        pad_size = 8192 - len(x1)%8192
        padded_noisy = np.pad(x1, (0, pad_size), 'constant', constant_values=(0, 0))
        x2 = model(torch.tensor(padded_noisy).float().view(1,1,-1).to(device)).cpu().detach().numpy().reshape(-1)[:len(clean)]
        
        torchaudio.save(os.path.join(x2_path, file), torch.tensor(x2).view(1,-1), 16000)
        
        # break

 55%|█████▌    | 461/835 [01:02<00:50,  7.35it/s]


KeyboardInterrupt: 